In [25]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import random
import re
import json

In [26]:
# 读取中国省、直辖市、自治区列表。因为企查查不含港澳台数据，所以这里没有港澳台。
with open("provinces.json", 'r', encoding="utf-8") as f:
    provinces = json.load(f)

In [27]:
# 由于selenium是通过driver去操作浏览器的，所以我们需要对应浏览器的driver对象
driver = webdriver.Chrome()

第一次使用webdriver需要登陆，但是登录的验证操作种类很多，不方便一一破解，所以选择在当前webdriver打开的浏览器上手动登录一次后再访问需要的URL。

In [28]:
driver.get('https://www.qichacha.com/user_login')
# phone_number = driver.find_element(By.NAME, 'phone-number') # 不止一个的name是phone-number，所以不能直接根据名称来搜索
phone_number = driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[2]/div/div[1]/div[3]/form/div[1]/input')
password = driver.find_element(By.NAME, 'password')
pn = 12345678910
pw = '12345678'
phone_number.send_keys(pn)
password.send_keys(pw)
login_btn = driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[2]/div/div[1]/div[3]/form/div[4]/button/strong')
login_btn.click()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[1]/div[2]/div[2]/div/div[1]/div[3]/form/div[1]/input"}
  (Session info: chrome=125.0.6422.142); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6D3FF1F52+60322]
	(No symbol) [0x00007FF6D3F6CEC9]
	(No symbol) [0x00007FF6D3E27EBA]
	(No symbol) [0x00007FF6D3E77676]
	(No symbol) [0x00007FF6D3E7773C]
	(No symbol) [0x00007FF6D3EBE967]
	(No symbol) [0x00007FF6D3E9C25F]
	(No symbol) [0x00007FF6D3EBBC80]
	(No symbol) [0x00007FF6D3E9BFC3]
	(No symbol) [0x00007FF6D3E69617]
	(No symbol) [0x00007FF6D3E6A211]
	GetHandleVerifier [0x00007FF6D43094AD+3301629]
	GetHandleVerifier [0x00007FF6D43536D3+3605283]
	GetHandleVerifier [0x00007FF6D4349450+3563680]
	GetHandleVerifier [0x00007FF6D40A4326+790390]
	(No symbol) [0x00007FF6D3F7750F]
	(No symbol) [0

上面这个Python块会报错，手动登录qcc即可，然后运行后面的Python块

In [29]:
driver.get("https://www.qcc.com/web/search/advance?hasState=true")

为确保数据已经返回，每两个页面操作之间都必须有延时

In [35]:
# 操作时间间隔上下界
t_min = 1
t_max = 2

# 数据存储路径
data_path = 'data/'

for province in provinces:
    # 输入框输入城市
    # 每次刷新后输入框节点要重新获取
    search_input = driver.find_element(By.XPATH, '/html/body/div/div[2]/div/div[2]/div[2]/div[2]/div[2]/div[2]/div[1]/div/input')
    search_input.clear()# 输入框清零
    search_input.send_keys(province)
    search_input.click() # 确保出现勾选框
    time.sleep(1) # 等待数据返回

    parent_node_xpath = '/html/body/div/div[2]/div/div[2]/div[2]/div[2]/div[2]/div[2]/div[2]/div/div/div/div[1]/div[2]/ul/div[1]'
    parent_node = driver.find_element(By.XPATH, parent_node_xpath)
    time.sleep(1) # 等待数据返回

    # 获取地级市子节点
    child_nodes = parent_node.find_elements(By.XPATH, './/label/span[1]/input')
    time.sleep(5) # 这里延时要长，不然城市列表可能没有刷新
    
    city_counts = {}
    # 逐个点击这些子节点
    for node in child_nodes:
        # 勾选
        node.click()
        time.sleep(random.uniform(t_min, t_max)) # 操作太快会被发现

        # 城市名
        city_name = node.accessible_name
        # 获取结果数
        count_node = driver.find_element(By.CLASS_NAME, 'filter-search-left')
        count_text = count_node.text
        # 使用正则表达式提取数字
        numbers = re.findall(r'\d+', count_text)
        # 将数字列表拼接为一个完整的数字字符串，并转为int类型
        count = int(''.join(numbers))

        city_counts[city_name] = count

        print(city_counts)

        # 取消勾选
        node.click()
        time.sleep(random.uniform(t_min, t_max))

    # 数据保存
    with open(data_path+province+'.json', "w", encoding="utf-8") as f:
        json.dump(city_counts, f, ensure_ascii=False, indent=4)
    
    # 清除搜索框
    city_clear = driver.find_element(By.XPATH, '/html/body/div/div[2]/div/div[2]/div[2]/div[2]/div[2]/div[2]/div[1]/div/i[2]')
    city_clear.click()
    time.sleep(1) # 等待数据返回

    driver.refresh() # 刷新浏览器，减少浏览器bug
    time.sleep(3) # 等待数据返回


{'东城区': 211102}
{'东城区': 211102, '西城区': 225287}
{'东城区': 211102, '西城区': 225287, '朝阳区': 950697}
{'东城区': 211102, '西城区': 225287, '朝阳区': 950697, '丰台区': 515167}
{'东城区': 211102, '西城区': 225287, '朝阳区': 950697, '丰台区': 515167, '石景山区': 139138}
{'东城区': 211102, '西城区': 225287, '朝阳区': 950697, '丰台区': 515167, '石景山区': 139138, '海淀区': 734024}
{'东城区': 211102, '西城区': 225287, '朝阳区': 950697, '丰台区': 515167, '石景山区': 139138, '海淀区': 734024, '门头沟区': 132981}
{'东城区': 211102, '西城区': 225287, '朝阳区': 950697, '丰台区': 515167, '石景山区': 139138, '海淀区': 734024, '门头沟区': 132981, '房山区': 292411}
{'东城区': 211102, '西城区': 225287, '朝阳区': 950697, '丰台区': 515167, '石景山区': 139138, '海淀区': 734024, '门头沟区': 132981, '房山区': 292411, '通州区': 376413}
{'东城区': 211102, '西城区': 225287, '朝阳区': 950697, '丰台区': 515167, '石景山区': 139138, '海淀区': 734024, '门头沟区': 132981, '房山区': 292411, '通州区': 376413, '顺义区': 253890}
{'东城区': 211102, '西城区': 225287, '朝阳区': 950697, '丰台区': 515167, '石景山区': 139138, '海淀区': 734024, '门头沟区': 132981, '房山区': 292411, '通州区': 376413, '顺义区': 253890, '昌